In [2]:
try:
    import google.colab
    import torch
    COLAB = True
    print("Note: using Google CoLab")
    
    has_mps = torch.backends.mps.is_built()
    device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
except:
    import torch
    print("Note: not using Google CoLab")
    COLAB = False
    
    has_mps = torch.backends.mps.is_built()
    device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
print(f"Using device: {device}")

Note: not using Google CoLab
Using device: cpu


Import data

In [141]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("data.csv",sep=',', header=0,index_col=False)
df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace inf with NaN
df.fillna(method='ffill', inplace=True)  # Forward fill NaNs


num_setups = 3
split_time = 0.6
train_data = df[df['time'] < split_time]  
test_data = df[df['time'] >= split_time]  

print(df.head())
print('train shape: ', train_data.shape)
print('test shape:  ', test_data.shape)

       time         u         x
0  0.000000  0.250000  0.125000
1  0.111111  0.264706  0.243464
2  0.222222  0.281250  0.362847
3  0.333333  0.300000  0.483333
4  0.444444  0.321429  0.605159
train shape:  (18, 3)
test shape:   (12, 3)


In [115]:
# Select position and control columns
train_array = train_data[['x', 'u']]
test_array = test_data[['x', 'u']]

# Reshape test and train arrays
train_array = train_array.to_numpy().reshape(num_setups, train_array.shape[0]//num_setups, 2) 
test_array = test_array.to_numpy().reshape(num_setups, test_array.shape[0]//num_setups, 2)

print('train shape: ', train_array.shape)
print('test shape:  ', test_array.shape)

train shape:  (3, 6, 2)
test shape:   (3, 4, 2)


In [148]:
def create_sequences(data, sequence_length):
    inputs = []
    targets = []

    for setup in data:
        inputs.append(setup[:sequence_length])
        targets.append(setup[1:sequence_length+1,0])

    return np.array(inputs), np.array(targets)

# Create sequences
x_train, y_train = create_sequences(train_array,train_array.shape[1]-1)
x_test, y_test = create_sequences(test_array, test_array.shape[1]-1)

print('x train shape: ', x_train.shape)
print('y train shape: ', y_train.shape)
print('x test shape: ', x_test.shape)
print('y test shape: ', y_test.shape)

x train shape:  (3, 5, 2)
y train shape:  (3, 5)
x test shape:  (3, 3, 2)
y test shape:  (3, 3)


In [149]:
# Convert to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

print('x train shape: ', x_train_tensor.shape)
print('y train shape: ', y_train_tensor.shape)
print('x test shape: ', x_test_tensor.shape)
print('y test shape: ', y_test_tensor.shape)

x train shape:  torch.Size([3, 5, 2])
y train shape:  torch.Size([3, 5])
x test shape:  torch.Size([3, 3, 2])
y test shape:  torch.Size([3, 3])


In [150]:
# Create TensorDatasets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(train_dataset)
print(train_dataset)

In [164]:
# Positional Encoding for Transformer
class DecoderPositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=20):
        super(DecoderPositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


# Model definition using Transformer
class DecoderTransformerModel(nn.Module):
    global num_timesteps
    # attention head = 4
    # encode layers = 2
    # input seq of size 1 (1 token)
    # embedding dim = 64
    # dropout = 0.2 --> every training step each neuron has 20% chance of not contributing (to the forward pass and  being updated during backward pass)
    def __init__(self, input_dim=num_timesteps, d_model=2, nhead=2, num_layers=1, dropout=0.2):
        super(DecoderTransformerModel, self).__init__()

        # info
        self.model_type = "Transformer"
        self.d_model = d_model

        # define layers
        #self.embedding = nn.Linear(input_dim, d_model) #(input eg [128,20] with nn.Linear(20,50) --> output [128,50])
        self.pos_encoder = DecoderPositionalEncoding(d_model, dropout)
        decoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=10, dropout=dropout, activation="relu", layer_norm_eps=1e-05, batch_first=False, norm_first=False, bias=True, device=device)
        self.transformer_decoder = nn.TransformerEncoder(decoder_layers, num_layers,mask_check=True)
        self.linear = nn.Linear(d_model, 1)

    def forward(self, x, decoder_mask=None):
        #x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer_decoder(x, decoder_mask)
        x = self.linear(x[:, -1, :]) #the output from the last token of the sequence is passed through a linear layer to produce the final model output.
        return x

    def get_decoder_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one element more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        #print('maask', mask)
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]

        return mask

decoder_model = DecoderTransformerModel().to(device)

In [167]:
# Train the decoder model
'''
On CPU train time = 9 min 45s
On google collab GPU the model is trained in a matter of seconds
'''
from torch.optim.lr_scheduler import ReduceLROnPlateau
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(decoder_model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 1000
early_stop_count = 0
min_val_loss = float('inf')
i = 0
for epoch in range(epochs):
    decoder_model.train()
    for batch in train_dataloader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Get mask to mask out the next words
        sequence_length = x_batch.size(0)
        decoder_mask = decoder_model.get_decoder_mask(sequence_length).to(device)

        optimizer.zero_grad()
        outputs = decoder_model(x_batch,decoder_mask)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    decoder_model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_dataloader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = decoder_model(x_batch,decoder_mask)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)

    if val_loss <= min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 20:
        print("Stopping: Loss increasing")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")

Epoch 1/1000, Validation Loss: 3.3047
Epoch 2/1000, Validation Loss: 3.3111
Epoch 3/1000, Validation Loss: 3.3180
Epoch 4/1000, Validation Loss: 3.3250
Epoch 5/1000, Validation Loss: 3.3321
Epoch 6/1000, Validation Loss: 3.3374
Epoch 7/1000, Validation Loss: 3.3428
Epoch 8/1000, Validation Loss: 3.3481
Epoch 9/1000, Validation Loss: 3.3536
Epoch 10/1000, Validation Loss: 3.3566
Epoch 11/1000, Validation Loss: 3.3592
Epoch 12/1000, Validation Loss: 3.3616
Epoch 13/1000, Validation Loss: 3.3639
Epoch 14/1000, Validation Loss: 3.3649
Epoch 15/1000, Validation Loss: 3.3659
Epoch 16/1000, Validation Loss: 3.3668
Epoch 17/1000, Validation Loss: 3.3678
Epoch 18/1000, Validation Loss: 3.3682
Epoch 19/1000, Validation Loss: 3.3687
Epoch 20/1000, Validation Loss: 3.3691
Stopping!: Loss increasing


In [168]:
# Evaluation of decodr model
from torch.nn.functional import mse_loss

decoder_model.eval()
all_predictions = []
all_targets = []

with torch.no_grad():
    for batch in test_dataloader:
        x_batch, y_batch = batch
        output = decoder_model(x_batch)
        
        # Store predictions and targets
        all_predictions.append(output)
        all_targets.append(y_batch)

# Concatenate all batches
all_predictions = torch.cat(all_predictions, dim=0)
all_targets = torch.cat(all_targets, dim=0)

# Calculate MSE and then take the square root to get RMSE
mse = mse_loss(all_predictions, all_targets)
rmse = np.sqrt(mse.item())

print(f'Test RMSE: {rmse}')

#RMSE: The root mean square error (RMSE) measures the average difference between a statistical model’s predicted values and the actual values. 
# Mathematically, it is the standard deviation of the residuals. It is a non-standardized (0->inf) metric of a model's goodness of fit. 
# It uses the units of the dependent variable and gives a direct assessment of prediction precision. It is sensitive to outliers.
# https://statisticsbyjim.com/regression/root-mean-square-error-rmse/

#R-squared: A standardized (0->1) goodness of fit metric which is unitless. It is sensitive to outliers (see adjusted R-squared).
#R-squared is the percentage of the response variable variation that is explained by a linear model (how close the data are to the fitted regression line).
# R-squared = 1 - (SSR / SST)
# The sum squared regression (SSR) is the sum of the squared differences between the predicted values and the actual values.
# The total sum of squares (SST) represents the sum of the squares of the differences between each actual value and the overall mean of the data set.

# Check back design of experiments course

Test RMSE: 1.83564996293901


C:\Users\sian_\AppData\Local\Temp\ipykernel_27948\2377093126.py:22: UserWarning: Using a target size (torch.Size([3, 3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse = mse_loss(all_predictions, all_targets)
